# Module 2 Project - Housing Price Regression

## Scope
I will be building a multivariate regression on the KC Housing Dataset with the goal of predicting house sale price as accurately as possible. This exercise will be completed in the context of a private equity / real estate investment firm looking for diligence and valuation support. Ideally, this regression will help identify statistically significant variables to target during due diligence processes to ensure time is spent researching aspects of a house that are meaningful to price. Additionally, if the data supports it, firms will be able to leverage this data to determine which potential future projects might be correlated with higher sales prices to guide renovation work. Ultimately, using the tool for buyers and sellers, this tool can be used to minimize risk and help improve return realization. 

I will aim to answer the following three primary questions:
1. What does the current KC housing market look like and what do the majority of houses have in common
2. Are there specific times (years, months, days) that might be related with a higher sale price?
3. Are provided rankings (grade, condition, etc.) trustworthy and are they actually relevant to predicting sale price?

## Overview of Process:
General process steps are as followed:
1. Load dataset and handle data issues (missing values, weird values, column data types, etc.)
2. Identify continuous, ordinal categorical, and non-ordinal categorical variables
3. Drop continuous variables that do not meet linearity requirements
4. Handle multi-collinearity amongst predictors
5. Handle categorical variables - treat ordinal categorical variables as single columns and encode non-ordinal categorical columns
6. Use stepwise selection to select features that meet p-value thresholds 
7. Run baseline model with these features 
8. Evaluate regression diagnostics and check assumptions
9. Continue preprocessing / transforming to improve results